In [33]:
#Imports, file discovery, helpers

import numpy as np, pandas as pd
from pathlib import Path

# --- file discovery ---
FILES = {
    "feature_view": Path("feature_view.csv"),
    "tradelines": Path("tradelines.csv"),
    "scores": Path("scores.csv"),
    "fraud_train": Path("fraud_train_cleaned_data.csv"),
    "fraud_test": Path("fraud_test_cleaned_data.csv"),
}
def exists(p):
    try: return p.exists()
    except: return False

for k,p in FILES.items():
    print(f"{k:12s} -> {'FOUND' if exists(p) else 'missing'}: {p}")

# --- id resolution & utilities ---
ID_CANDIDATES = ["customer_id","consumer_id","person_id","client_id","cc_num","card_id","account_id","cust_id"]

def resolve_id_column(df, preferred=None):
    if preferred and preferred in df.columns:
        return preferred
    for c in ID_CANDIDATES:
        if c in df.columns:
            return c
    for c in df.columns:  # last-resort heuristic
        cl = c.lower()
        if 'id' in cl or 'cc_num' in cl:
            return c
    raise KeyError("No suitable customer id column found.")

def entropy_from_counts(counts):
    total = sum(counts)
    if total <= 0: return 0.0
    p = np.array(counts, dtype=float) / total
    return float(-(p[p>0] * np.log2(p[p>0])).sum())



feature_view -> FOUND: feature_view.csv
tradelines   -> FOUND: tradelines.csv
scores       -> FOUND: scores.csv
fraud_train  -> FOUND: fraud_train_cleaned_data.csv
fraud_test   -> FOUND: fraud_test_cleaned_data.csv


In [34]:
#Load the datasets that are present

dfs = {}

if exists(FILES["feature_view"]):
    dfs["feature_view"] = pd.read_csv(FILES["feature_view"])

if exists(FILES["tradelines"]):
    dfs["tradelines"] = pd.read_csv(FILES["tradelines"])

if exists(FILES["scores"]):
    dfs["scores"] = pd.read_csv(FILES["scores"])

if exists(FILES["fraud_train"]):
    try:
        dfs["fraud_train"] = pd.read_csv(FILES["fraud_train"], parse_dates=["trans_date_trans_time"])
    except Exception:
        dfs["fraud_train"] = pd.read_csv(FILES["fraud_train"])

if exists(FILES["fraud_test"]):
    try:
        dfs["fraud_test"] = pd.read_csv(FILES["fraud_test"], parse_dates=["trans_date_trans_time"])
    except Exception:
        dfs["fraud_test"] = pd.read_csv(FILES["fraud_test"])

for name, df in dfs.items():
    print(f"{name:12s}", df.shape)
    display(df.head(2))


feature_view (1200, 22)


,consumer_id,dob,primary_state,fico8,vantage4,score_date,tradelines_total,tradelines_open,revolving_limit_sum,revolving_balance_sum,...,delinq_24mo_count,worst_delinq_24mo,inquiries_12mo_hard,inquiries_12mo_soft,has_collection,collection_balance_total,has_bankruptcy,bankruptcy_most_recent,months_since_oldest_account,avg_account_age_months
0,C0001,1972-01-24,NJ,572,663,2025-10-05,4,3,60000,40116.47,...,19,3,1,0,0,0,0,NaN,144,98.2
1,C0002,1980-04-05,FL,546,729,2025-10-05,4,0,30000,17824.57,...,17,3,0,2,0,0,0,NaN,105,82.5


tradelines   (3531, 14)


,consumer_id,tradeline_id,subscriber_name,subscriber_code,account_type,opened,closed,credit_limit,high_credit,balance,past_due,payment_status,terms,months_reviewed
0,C0001,T00001,Bank_115,SUB62709,installment,2016-10-07,NaN,0,22000,2991.38,500,current,R,108
1,C0001,T00002,Bank_323,SUB63883,revolving,2020-01-13,2022-01-12,30000,30000,24074.44,500,current,R,69


scores       (2400, 5)


,consumer_id,model,score,score_date,reason_text
0,C0001,FICO8,572,2025-10-05,Delinquencies in past 24 months
1,C0001,VantageScore4,663,2025-10-05,Low utilization; long history


fraud_train  (430313, 23)


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495.0,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1.325376e+09,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149.0,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1.325376e+09,49.159047,-118.186462,0.0


fraud_test   (555719, 23)


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0


In [35]:
#Tradelines → numeric features

tradelines_feats = pd.DataFrame()

if "tradelines" in dfs:
    tl = dfs["tradelines"].copy()
    cid = resolve_id_column(tl)
    L = tl.columns.str.lower()

    def match(*names):
        for n in names:
            if n in L.values:
                return tl.columns[L==n][0]
        return None

    # balances / limits
    bal = match("revolving_balance_sum","revolving_balance","balance","total_balance","current_balance")
    lim = match("revolving_limit_sum","revolving_limit","credit_limit","total_limit","limit")

    # payment performance
    onp = match("on_time_payments","ontime_payments","payments_on_time","paid_on_time")
    tot = match("total_payments","num_payments","payments_total")

    # delinquencies & collections
    d30 = match("delinq_30","delinquency_30","late_30")
    d60 = match("delinq_60","delinquency_60","late_60")
    d90 = match("delinq_90","delinquency_90","late_90")
    chg = match("charge_offs","chargeoff_count","default_count")
    col = match("collection_count","collections")

    g = tl.groupby(cid)
    tradelines_feats["customer_id"] = g.size().index

    tradelines_feats["on_time_payment_ratio"] = (
        (g[onp].sum() / g[tot].sum()).replace([np.inf, np.nan], 0.0).values if (onp and tot) else 0.0
    )

    if bal and lim:
        B = g[bal].sum()
        Lm = g[lim].sum().replace(0, np.nan)
        tradelines_feats["utilization_ratio"] = (B/Lm).fillna(0.0).clip(0, 5.0).values
        tradelines_feats["total_limit"] = Lm.fillna(0.0).values
        tradelines_feats["total_balance"] = B.fillna(0.0).values
    else:
        tradelines_feats[["utilization_ratio","total_limit","total_balance"]] = 0.0

    for out, src in {
        "delinq_30": d30, "delinq_60": d60, "delinq_90": d90,
        "charge_off": chg, "collections": col
    }.items():
        tradelines_feats[out] = g[src].sum().values if src else 0
else:
    print("tradelines.csv not found → skipping tradeline features")

display(tradelines_feats.head())


,customer_id,on_time_payment_ratio,utilization_ratio,total_limit,total_balance,delinq_30,delinq_60,delinq_90,charge_off,collections
0,C0001,0.0,5.000000,60000.0,344960.88,0,0,0,0,0
1,C0002,0.0,5.000000,30000.0,489705.83,0,0,0,0,0
2,C0003,0.0,5.000000,30000.0,188658.81,0,0,0,0,0
3,C0004,0.0,0.848038,15000.0,12720.57,0,0,0,0,0
4,C0005,0.0,5.000000,1000.0,169228.40,0,0,0,0,0


In [36]:
#Cash-Flow & (c) Behavioral → from transactions

cash_behav_feats = pd.DataFrame()

if "fraud_train" in dfs:
    tx = dfs["fraud_train"].copy()
    cid = resolve_id_column(tx)

    # timestamp & month
    if "trans_date_trans_time" in tx.columns:
        tx["tx_time"] = pd.to_datetime(tx["trans_date_trans_time"], errors="coerce")
    elif "timestamp" in tx.columns:
        tx["tx_time"] = pd.to_datetime(tx["timestamp"], errors="coerce")
    else:
        tx["tx_time"] = pd.NaT
    tx["tx_month"] = tx["tx_time"].dt.to_period("M").dt.to_timestamp()

    # amount column
    amt_col = None
    for c in ["amt","amount","transaction_amount"]:
        if c in tx.columns: amt_col = c; break
    if amt_col is None:
        raise KeyError("No amount column found in transactions")

    # category for entropy / diversity
    cat_col = "category" if "category" in tx.columns else None

    # monthly customer aggregates
    magg = tx.groupby([cid,"tx_month"]).agg(
        monthly_spend = (amt_col,"sum"),
        tx_count      = (amt_col,"count")
    ).reset_index()
    gc = magg.groupby(cid)

    cash_behav_feats["customer_id"] = gc.size().index
    cash_behav_feats["avg_monthly_spend"] = gc["monthly_spend"].mean().values
    cash_behav_feats["monthly_spend_volatility"] = (
        (gc["monthly_spend"].std() / gc["monthly_spend"].mean())
        .replace([np.inf, np.nan], 0.0).values
    )
    cash_behav_feats["active_months"] = gc.size().values

    # trend proxy: last-first over periods
    def trend(x):
        x = x.dropna()
        if len(x) < 2: return 0.0
        return (x.iloc[-1] - x.iloc[0]) / max(1, len(x)-1)
    cash_behav_feats["net_cashflow_trend"] = gc["monthly_spend"].apply(trend).values

    # entropy & merchant diversity
    if cat_col:
        cat_counts = tx.groupby([cid,cat_col]).size().reset_index(name="n")
        ent = cat_counts.groupby(cid)["n"].apply(lambda v: entropy_from_counts(v.tolist()))
        ent = ent.reindex(gc.size().index).fillna(0.0)
        cash_behav_feats["expense_entropy"] = ent.values
        cash_behav_feats["merchant_diversity"] = (
            tx.groupby(cid)[cat_col].nunique().reindex(gc.size().index).fillna(0).astype(int).values
        )
    else:
        cash_behav_feats[["expense_entropy","merchant_diversity"]] = 0.0

    # payment day regularity (std of day-of-month of all txs; higher std = less regular)
    tx["day"] = tx["tx_time"].dt.day
    def day_std(s):
        s = s.dropna()
        return s.std() if len(s) >= 3 else 31.0
    cash_behav_feats["payment_day_regular_std"] = (
        tx.groupby(cid)["day"].apply(day_std).reindex(gc.size().index).fillna(31.0).values
    )

    # cash intensity if categories include "cash"
    if cat_col:
        mask = tx[cat_col].str.contains("cash", case=False, na=False)
        tot = tx.groupby(cid)[amt_col].sum()
        cash_amt = tx.loc[mask].groupby(cid)[amt_col].sum()
        cash_behav_feats["cash_intensity"] = (cash_amt/tot).reindex(tot.index).fillna(0.0).values
    else:
        cash_behav_feats["cash_intensity"] = 0.0
else:
    print("fraud_train_cleaned_data.csv not found → skipping cash-flow & behavioral features")

display(cash_behav_feats.head())


,customer_id,avg_monthly_spend,monthly_spend_volatility,active_months,net_cashflow_trend,expense_entropy,merchant_diversity,payment_day_regular_std,cash_intensity
0,60416207185,4646.335714,0.588120,7,-133.181667,3.656457,14,8.731713,0.0
1,60422928733,5640.240000,0.688654,7,-48.146667,3.698438,14,8.513437,0.0
2,60423098130,1940.074286,0.942232,7,-34.521667,3.605913,14,8.631288,0.0
3,60427851591,3709.182857,0.972623,7,-204.763333,3.687437,14,8.393709,0.0
4,60487002085,1168.207143,0.477481,7,-90.586667,3.625051,14,8.926995,0.0


In [37]:
#Credit Mix & AAoA

creditmix_feats = pd.DataFrame()

source = "tradelines" if "tradelines" in dfs else ("feature_view" if "feature_view" in dfs else None)
if source:
    df = dfs[source].copy()
    cid = resolve_id_column(df)
    out = pd.DataFrame({"customer_id": df.groupby(cid).size().index})
    L = df.columns.str.lower()

    def match(*names):
        for n in names:
            if n in L.values:
                return df.columns[L==n][0]
        return None

    rev = match("revolving_accounts","num_revolving_accounts","revolving_count")
    ins = match("installment_loans","num_installment_loans","installment_count","num_installment_accounts")
    mor = match("mortgage_count","num_mortgages","mortgages")
    age = match("avg_account_age_months","average_age_months","aaoa_months","avg_account_age")

    out["num_revolving_accounts"] = df.groupby(cid)[rev].sum().values if rev else 0
    out["num_installment_loans"] = df.groupby(cid)[ins].sum().values if ins else 0
    out["num_mortgages"]        = df.groupby(cid)[mor].sum().values if mor else 0
    out["aaoa_months"]          = df.groupby(cid)[age].mean().values if age else 0.0

    creditmix_feats = out
else:
    print("No tradelines/feature_view → skipping credit mix")

display(creditmix_feats.head())


,customer_id,num_revolving_accounts,num_installment_loans,num_mortgages,aaoa_months
0,C0001,0,0,0,0.0
1,C0002,0,0,0,0.0
2,C0003,0,0,0,0.0
3,C0004,0,0,0,0.0
4,C0005,0,0,0,0.0


In [38]:
#Inquiries / New Credit
inq_feats = pd.DataFrame()

source = "feature_view" if "feature_view" in dfs else ("tradelines" if "tradelines" in dfs else None)
if source:
    df = dfs[source].copy()
    cid = resolve_id_column(df)
    out = pd.DataFrame({"customer_id": df.groupby(cid).size().index})
    L = df.columns.str.lower()

    def match(*names):
        for n in names:
            if n in L.values:
                return df.columns[L==n][0]
        return None

    inq = match("inquiries_6m","inquiries_3m","hard_inquiries_12m","inquiries_12m")
    new = match("new_accounts_6m","opened_accounts_6m","new_accounts_12m")
    tlc = match("total_tradelines","tradelines_total","num_accounts")

    out["inquiries_recent"]    = df.groupby(cid)[inq].sum().values if inq else 0
    out["new_accounts_recent"] = df.groupby(cid)[new].sum().values if new else 0
    if tlc:
        cnt = df.groupby(cid)[tlc].max()
        out["thin_file_flag"] = (cnt < 3).astype(int).reindex(out["customer_id"]).fillna(1).values
    else:
        out["thin_file_flag"] = 1

    inq_feats = out
else:
    print("No feature_view/tradelines → default thin-file=1, inquiries=0")

display(inq_feats.head())


,customer_id,inquiries_recent,new_accounts_recent,thin_file_flag
0,C0001,0,0,0
1,C0002,0,0,0
2,C0003,0,0,0
3,C0004,0,0,0
4,C0005,0,0,0


In [39]:
#Geo-Economic placeholders
geo_feats = pd.DataFrame()

if "fraud_train" in dfs and "city_pop" in dfs["fraud_train"].columns:
    df = dfs["fraud_train"]
    cid = resolve_id_column(df)
    base = pd.DataFrame({"customer_id": df.groupby(cid).size().index})
    geo_feats = (
        base.join(df.groupby(cid)["city_pop"].mean(), on="customer_id")
            .rename(columns={"city_pop":"avg_city_pop"})
            .fillna(0.0)
    )
else:
    print("Geo features not available (no city_pop).")

display(geo_feats.head())


,customer_id,avg_city_pop
0,60416207185,1645.0
1,60422928733,46944.0
2,60423098130,83.0
3,60427851591,2142.0
4,60487002085,233060.0


In [40]:
#merge all the blocks and save
blocks = []
for name, block in [
    ("tradelines", tradelines_feats),
    ("cash_behav", cash_behav_feats),
    ("creditmix", creditmix_feats),
    ("inquiries", inq_feats),
    ("geo", geo_feats),
]:
    if isinstance(block, pd.DataFrame) and not block.empty:
        print(f"Adding {name}: {block.shape}")
        blocks.append(block.set_index("customer_id"))

features = pd.concat(blocks, axis=1).reset_index() if blocks else pd.DataFrame(columns=["customer_id"])
print("Final feature table:", features.shape)
display(features.head())

features.to_csv("engineered_features.csv", index=False)
print("Saved -> engineered_features.csv")


Adding tradelines: (1200, 10)
Adding cash_behav: (937, 9)
Adding creditmix: (1200, 5)
Adding inquiries: (1200, 4)
Adding geo: (937, 2)
Final feature table: (2137, 26)


,customer_id,on_time_payment_ratio,utilization_ratio,total_limit,total_balance,delinq_30,delinq_60,delinq_90,charge_off,collections,...,payment_day_regular_std,cash_intensity,num_revolving_accounts,num_installment_loans,num_mortgages,aaoa_months,inquiries_recent,new_accounts_recent,thin_file_flag,avg_city_pop
0,C0001,0.0,5.000000,60000.0,344960.88,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,C0002,0.0,5.000000,30000.0,489705.83,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,C0003,0.0,5.000000,30000.0,188658.81,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,C0004,0.0,0.848038,15000.0,12720.57,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,C0005,0.0,5.000000,1000.0,169228.40,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


Saved -> engineered_features.csv


In [41]:
df = pd.read_csv("engineered_features.csv")
df.describe().T


,count,mean,std,min,25%,50%,75%,max
on_time_payment_ratio,1200.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
utilization_ratio,1200.0,2.382582,2.232775,0.000000,0.000000,1.471939,5.000000,5.000000e+00
total_limit,1200.0,11945.000000,14762.234857,0.000000,0.000000,5000.000000,20000.000000,9.000000e+04
total_balance,1200.0,155281.643608,158483.531156,381.370000,22479.970000,107467.710000,245644.460000,1.021558e+06
delinq_30,1200.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
delinq_60,1200.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
delinq_90,1200.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
charge_off,1200.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
collections,1200.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
avg_monthly_spend,937.0,4781.075703,2726.806530,965.060000,2674.621429,4401.654286,6280.092857,1.463331e+04
